<a href="https://colab.research.google.com/github/siddhant-parmar/CMPE_255_Group_4/blob/main/Classifier_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np 

review_data = pd.read_csv("/content/drive/My Drive/cmpe255/yelp_review.csv")
busi_data = pd.read_csv("/content/drive/My Drive/cmpe255/yelp_business.csv")

In [4]:
review_data = review_data[['business_id', 'text', 'stars']]
review_data.head()

,business_id,text,stars
0,AEx2SYEUJmTxVVB18LlCwA,Super simple place but amazing nonetheless. It...,5
1,VR6GpWIda3SfvPC-lg9H3w,Small unassuming place that changes their menu...,5
2,CKC0-MOWMqoeWf6s-szl8g,Lester's is located in a beautiful neighborhoo...,5
3,ACFtxLv8pGrrxMm6EgjreA,Love coming here. Yes the place always needs t...,4
4,s2I_Ni76bjJNK9yG60iD-Q,Had their chocolate almond croissant and it wa...,4


In [5]:
busi_data = busi_data[busi_data.categories.str.contains('restaurant|food', na=False)]
busi_data = busi_data[['business_id', 'name']]
busi_data.head()

,business_id,name
44,BnuzcebyB1AfxH0kjNWqSg,"""Carrabba's Italian Grill"""
101,VdlPZg2NAu8t8GkdbPLecg,"""Tandoori Times Indian Bistro"""
107,QkG3KUXwqZBW18A9k1xqCA,"""Red Lobster"""
163,xMP2oDrKFqyUrN0Uww85EQ,"""Dockside Seafood Market"""
274,dPxZI9lrKTl5dvFfnb1_Ig,"""Trattoria Italia"""


In [6]:
review_data['business_id'] = review_data['business_id'].astype(str)
busi_data['business_id'] = busi_data['business_id'].astype(str)

data = review_data.merge(busi_data, on='business_id')
print(data.shape)
data.head()

(248709, 4)


,business_id,text,stars,name
0,RWGI8u00x5GghYCEzpYrIg,This place reminds of St Lawrence Market in To...,4,"""Marché Atwater"""
1,RWGI8u00x5GghYCEzpYrIg,A very quaint farmers market right by the wate...,4,"""Marché Atwater"""
2,RWGI8u00x5GghYCEzpYrIg,A must on any Montreal visit if nothing else b...,5,"""Marché Atwater"""
3,RWGI8u00x5GghYCEzpYrIg,"Marche Atwater is a cute little market, they h...",4,"""Marché Atwater"""
4,RWGI8u00x5GghYCEzpYrIg,Le Coin Gourmand carries the best store bought...,4,"""Marché Atwater"""


In [7]:
import nltk 
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

data['text'] = data['text'].astype(str)
#print(data.head())
data['text'] = data['text'].str.lower()
#print(data.head())

# stop word removal
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,business_id,text,stars,name
0,RWGI8u00x5GghYCEzpYrIg,place reminds st lawrence market toronto veggi...,4,"""Marché Atwater"""
1,RWGI8u00x5GghYCEzpYrIg,"quaint farmers market right waterfront, makes ...",4,"""Marché Atwater"""
2,RWGI8u00x5GghYCEzpYrIg,must montreal visit nothing else gawk longingl...,5,"""Marché Atwater"""
3,RWGI8u00x5GghYCEzpYrIg,"marche atwater cute little market, booths outs...",4,"""Marché Atwater"""
4,RWGI8u00x5GghYCEzpYrIg,le coin gourmand carries best store bought chi...,4,"""Marché Atwater"""


In [8]:
import string
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import WhitespaceTokenizer
import re

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
    
    
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    # lemmatize text
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))

    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    # print(wordnet_tagged)

    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    text = " ".join(lemmatized_sentence)
    return(text)


def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def clean_text(text):
    # remove html tags
    text = remove_html_tags(text)
    # lower text
    text = text.lower()
    # unicode characters
    text = text.encode('ascii', 'ignore').decode()
    # remove digits
    text = ''.join([i for i in text if not i.isdigit()])
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # remove stop words
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
data['text'] = data.text.apply(clean_text)
data.head()

,business_id,text,stars,name
0,RWGI8u00x5GghYCEzpYrIg,place reminds st lawrence market toronto veggi...,4,"""Marché Atwater"""
1,RWGI8u00x5GghYCEzpYrIg,quaint farmers market right waterfront makes g...,4,"""Marché Atwater"""
2,RWGI8u00x5GghYCEzpYrIg,must montreal visit nothing else gawk longingl...,5,"""Marché Atwater"""
3,RWGI8u00x5GghYCEzpYrIg,marche atwater cute little market booths outsi...,4,"""Marché Atwater"""
4,RWGI8u00x5GghYCEzpYrIg,le coin gourmand carries best store bought chi...,4,"""Marché Atwater"""


In [10]:
words = ["discomfort", "ache", "death", "allergy", "diarrhoea", "infection", "stomach", "loosemotion", "sick", "vomit", "nausea", "throw up", "threw up", "unhygienic", "pain", "flu", "stale", "poision", "motion", "upset", "indigestion", "acidity", "barf" ]

for w in words:
  w = lemmatize_text(w);

In [11]:
# creating a new feature for food poisoning based on words seen in review text
for index,row in data.iterrows() :
    if any(word in row['text'] for word in words) and row['stars'] < 3:
        data.loc[index,'FP'] = 1
    else:
        data.loc[index,'FP'] = 0


data.head()

,business_id,text,stars,name,FP
0,RWGI8u00x5GghYCEzpYrIg,place reminds st lawrence market toronto veggi...,4,"""Marché Atwater""",0.0
1,RWGI8u00x5GghYCEzpYrIg,quaint farmers market right waterfront makes g...,4,"""Marché Atwater""",0.0
2,RWGI8u00x5GghYCEzpYrIg,must montreal visit nothing else gawk longingl...,5,"""Marché Atwater""",0.0
3,RWGI8u00x5GghYCEzpYrIg,marche atwater cute little market booths outsi...,4,"""Marché Atwater""",0.0
4,RWGI8u00x5GghYCEzpYrIg,le coin gourmand carries best store bought chi...,4,"""Marché Atwater""",0.0


In [12]:
from sklearn.model_selection import train_test_split

# split the data into train and test set
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [13]:
train_X = train['text']   # '0' corresponds to Texts/Reviews
train_y = train['FP']   # '1' corresponds to Label (1 - positive and 0 - negative)
test_X = test['text'] 
test_y = test['FP']

In [16]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
t = time()  # not compulsory

# loading CountVectorizer
tf_vectorizer = CountVectorizer() # or term frequency

X_train_tf = tf_vectorizer.fit_transform(train_X)
duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

Time taken to extract features from training data : 12.334680 seconds
n_samples: 198967, n_features: 157032


In [18]:
t = time()
X_test_tf = tf_vectorizer.transform(test_X)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

Time taken to extract features from test data : 2.678580 seconds
n_samples: 49742, n_features: 157032


In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# build naive bayes classification model
t = time()

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)

train time: 0.146s


In [26]:
# predict the new document from the testing dataset
t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)

test_time = time() - t
print("Naive Bayes classifier")
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score_nb = metrics.accuracy_score(test_y, y_pred)
print("accuracy:   %0.3f" % score_nb)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['Positive', 'Negative']))

print("confusion matrix:")
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

Naive Bayes classifier
test time:  0.079s
accuracy:   0.974
              precision    recall  f1-score   support

    Positive       0.99      0.99      0.99     48807
    Negative       0.27      0.22      0.24       935

    accuracy                           0.97     49742
   macro avg       0.63      0.61      0.61     49742
weighted avg       0.97      0.97      0.97     49742

confusion matrix:
[[48231   576]
 [  727   208]]
------------------------------


In [22]:
# building knn classifier
from sklearn.neighbors import KNeighborsClassifier
t = time()

classifier = KNeighborsClassifier(n_neighbors=150, algorithm='auto')
classifier.fit(X_train_tf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)

KNeighborsClassifier(n_neighbors=150)

In [27]:
t = time()
predicted_labels = classifier.predict(X_test_tf)


In [28]:
# compute the performance measures
print("KNN classifier")
test_time = time() - t
print("test time:  %0.3fs" % test_time)
score_knn = metrics.accuracy_score(test_y, predicted_labels)
print("accuracy:   %0.3f" % score_knn)

print(metrics.classification_report(test_y, predicted_labels,
                                            target_names=['Positive', 'Negative']))

print("confusion matrix:")
print(metrics.confusion_matrix(test_y, predicted_labels))

print('------------------------------')

KNN classifier
test time:  667.505s
accuracy:   0.981
              precision    recall  f1-score   support

    Positive       0.98      1.00      0.99     48807
    Negative       0.00      0.00      0.00       935

    accuracy                           0.98     49742
   macro avg       0.49      0.50      0.50     49742
weighted avg       0.96      0.98      0.97     49742

confusion matrix:
[[48807     0]
 [  935     0]]
------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
